# Collaborative filtering recommender system

## Dataset description

In this assignment you will use Apache Spark to build simple movie recommendation system on big MovieLens 20M dataset (https://grouplens.org/datasets/movielens/). The dataset contains 20 million ratings for 27000 movies given by 138000 users. 

Dataset format: comma-separated values (https://en.wikipedia.org/wiki/CSV)

The first line is a header:
userId,movieId,rating,timestamp

_userId_, _movieId_ are integers representing user and movies identifiers correspondingly

_rating_ is a floating point number from 1.0 to 5.0

_timestamp_ is an integer but it won’t be used in the assignment :)

## Reading dataset

Init spark session

In [1]:
import os

os.environ['PYSPARK_PYTHON'] = '/opt/conda/bin/python3.6'
os.environ['PYSPARK_DRIVER_PYTHON'] = '/opt/conda/bin/python3.6'

In [2]:
from pyspark.sql import SparkSession

spark_session = (
    SparkSession
    .builder
    .enableHiveSupport()
    .appName("recSys")
    .master("local[8]")
    .getOrCreate()
)
# sc = spark_session.sparkContext

In [3]:
data = spark_session.read.load(
    "/data/movielens/ratings_100k.csv",
    format='com.databricks.spark.csv', 
    header=True, 
    inferSchema=True
).cache()

In [4]:
data.show()

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|     31|   2.5|1260759144|
|     1|   1029|   3.0|1260759179|
|     1|   1061|   3.0|1260759182|
|     1|   1129|   2.0|1260759185|
|     1|   1172|   4.0|1260759205|
|     1|   1263|   2.0|1260759151|
|     1|   1287|   2.0|1260759187|
|     1|   1293|   2.0|1260759148|
|     1|   1339|   3.5|1260759125|
|     1|   1343|   2.0|1260759131|
|     1|   1371|   2.5|1260759135|
|     1|   1405|   1.0|1260759203|
|     1|   1953|   4.0|1260759191|
|     1|   2105|   4.0|1260759139|
|     1|   2150|   3.0|1260759194|
|     1|   2193|   2.0|1260759198|
|     1|   2294|   2.0|1260759108|
|     1|   2455|   2.5|1260759113|
|     1|   2968|   1.0|1260759200|
|     1|   3671|   3.0|1260759117|
+------+-------+------+----------+
only showing top 20 rows



In this assignment you will use collaborative filtering approach for making predictions. To find latent vector representation for users and items we suggest you to use explicit ALS method. Refer to the Spark MLLib documentation for Python API details: https://spark.apache.org/docs/2.1.0/api/python/pyspark.mllib.html

Split dataset into three folds: on each iteration one fold will be used for testing and the other two folds will be used for training. Wrap necessary values with _Rating_ class.

In [5]:
from pyspark.mllib.recommendation import Rating

fold_count = 3

In [6]:
data.rdd.map(lambda row: Rating(row[0], row[1], row[2]))

PythonRDD[15] at RDD at PythonRDD.scala:48

In [7]:
split_df = data.randomSplit([1.] * fold_count)

Recommendations: to make your solution more efficient prepare RDDs for all the folds beforehand running the training process.

## Training ALS model

Now it is time to choose training parameters. It is recommended to set the rank equal to 20,
regularization term lambda equal to 0.01 and the number of iterations equal to 5.
Feel free to experiment and choose your own parameters to see how they influence the final score.

In [8]:
rank = 10
iterations = 15
lambda_ = 0.01

Train the explicit ALS model on two of three folds and evaluate its performance on remaining test fold. Performance evaluation should be done using the classic RMSE metric.

In [9]:
scores = []

In [10]:
from pyspark.mllib.recommendation import ALS

In [11]:
from math import sqrt

In [12]:
def rating_to_tuple(row):
    return (row.user, row.product), row.rating

def rate_als(train, test):
    train_data = train.rdd.map(lambda row: Rating(*row[:3]))
    test_data = test.rdd.map(lambda row: Rating(*row[:3]))
    # train model
    model = ALS.train(train_data, rank, iterations, lambda_)
    # make predictions
    predictions = model.predictAll(test_data.map(lambda row: (row.user, row.product)))
    # calc rmse
    test_predictions = (
        test_data.map(rating_to_tuple)
        .join(predictions.map(rating_to_tuple))
    )

    abs_frobenius_error = sqrt(test_predictions.map(lambda row: ((row[1][0] - row[1][1]) ** 2)).sum())
    frob_error_orig = sqrt(test_data.map(lambda row: row.rating ** 2).sum())

    return abs_frobenius_error / frob_error_orig


In [13]:
for i in range(fold_count):
    scores.append(rate_als(split_df[i], split_df[(i + 1) % fold_count].union(split_df[(i + 2) % fold_count])))

Average RMSE scores on all the three folds and output the result to stdout.
You could refer to publicly available benchmarks (e.g. http://mymedialite.net/examples/datasets.html)
to find out what score to expect.

In [14]:
print (scores)

[0.3856004653605672, 0.38056223292831776, 0.3926149015357845]


In [15]:
score = sum(scores) / len(scores)

In [16]:
score

0.3862591999415565